# Callin Switzer
### 6 June 2019
___
### - Train Dense, Feedforward Neural Network with Keras
### - Train with fully actuated system

___

In [7]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.ticker import ScalarFormatter
import matplotlib.colors as colors
import os
import pandas as pd
import time
from datetime import datetime
import sys
import seaborn as sns
import scipy.io
import subprocess
import winsound
import pickle
import glob
import sqlite3

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
print("TensorFlow successfully installed.")
if tf.test.is_built_with_cuda():
    print("The installed version of TensorFlow includes GPU support.")
print(sys.version, "\n")
now = datetime.now()
print("last run on " + str(now))

# define directories
baseDir = os.getcwd()
dataDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\PythonGeneratedData_twoTorque_2'
figDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\Figs'
dataOutput = r'D:/Dropbox/AcademiaDropbox/mothMachineLearning_dataAndFigs/DataOutput_twoTorque'
savedModels = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\savedModels'
if not os.path.exists(dataOutput):
    os.mkdir(dataOutput)
if not os.path.exists(savedModels):
    os.mkdir(savedModels)


TensorFlow successfully installed.
The installed version of TensorFlow includes GPU support.
3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)] 

last run on 2019-11-14 11:53:35.230955


In [8]:
# import
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.models import Model
from keras import regularizers

# Keras callcacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras.backend as K

# Make training and test set

In [9]:
# get table names in database
con1 = sqlite3.connect(os.path.join(dataDir, "twoTorqueData.db"))
cursorObj = con1.cursor()
res = cursorObj.execute("SELECT name FROM sqlite_master WHERE type='table';")
tableNames = [name[0] for name in res]
con1.close()
print(tableNames)

['test', 'train']


In [10]:
con1 = sqlite3.connect(os.path.join(dataDir, "twoTorqueData.db"))
trainDF = pd.read_sql_query("SELECT * FROM train", con1)
testDF = pd.read_sql_query("SELECT * FROM test", con1)
con1.close()

In [11]:
trainDF.head()

,x_0,xd_0,y_0,yd_0,theta_0,thetad_0,phi_0,phid_0,x_f,xd_f,y_f,yd_f,theta_f,thetad_f,phi_f,phid_f,F,alpha,tau0,tau_w
0,0.0,1473.201644,0.0,1276.409045,3.623862,-11.851729,1.211000,7.662472,29.866409,564.269592,21.336507,178.728114,-10.007981,-1511.665595,-11.460196,-1484.165029,41181.992912,1.987846,-632162.376646,-59087.944289
1,0.0,292.053762,0.0,329.129735,0.473141,-13.332132,0.030206,-24.548492,7.249558,-18.500791,4.674845,811.994233,10.193245,869.755012,8.614343,801.903597,25150.218788,3.741918,929029.039471,30635.419374
2,0.0,122.864494,0.0,-77.860175,2.080511,-23.745955,2.269162,13.577434,0.795119,2307.996013,-0.387013,-1210.224936,13.660000,2437.064176,13.518852,2393.363053,33176.564043,4.106501,495429.618543,92261.347215
3,0.0,321.464802,0.0,-185.568646,5.970777,17.674272,0.253450,-23.262211,3.218772,742.740898,-2.445868,-238.234357,7.546394,436.597363,2.136279,479.836917,371.601599,0.668810,-402592.572461,31282.236616
4,0.0,-74.555675,0.0,1242.566503,1.493441,7.679440,1.916887,10.558238,-1.249155,1452.849870,18.110278,808.550119,13.904002,1641.289100,13.440743,1565.438974,35874.696078,5.480043,929295.194744,44737.069384


In [12]:
testDF.head()

,x_0,xd_0,y_0,yd_0,theta_0,thetad_0,phi_0,phid_0,x_f,xd_f,y_f,yd_f,theta_f,thetad_f,phi_f,phid_f,F,alpha,tau0,tau_w
0,0.0,-479.406169,0.0,808.125168,1.074911,14.604692,1.610820,0.342567,-6.660783,606.471232,12.261791,621.515054,-8.236292,-1336.864852,-7.196589,-1285.116125,34939.199287,2.388669,-398848.635711,-52170.359464
1,0.0,730.738390,0.0,1220.929629,0.827197,-23.387488,4.311270,-0.881112,14.569797,-184.936022,24.523712,615.494066,17.904582,1735.428823,21.279341,1724.809150,3107.102381,1.483513,-25964.162554,91746.061771
2,0.0,-535.832562,0.0,11.137485,4.688616,23.204799,2.610169,4.804531,-10.027311,-496.451379,0.622225,34.844611,3.955242,-59.962069,2.982034,-5.562360,4516.937271,3.652675,-669707.091885,-1741.998064
3,0.0,175.278845,0.0,785.341568,5.317840,2.127405,2.094728,-20.938639,4.916500,-234.110480,13.478325,-17.188555,18.349930,1540.056108,15.515905,1555.480968,32299.290613,0.430122,-255553.079201,90618.166726
4,0.0,-817.253749,0.0,1121.004442,5.870598,3.003811,1.970784,5.114152,-16.960388,-378.720359,18.915203,935.576479,14.181347,839.473717,11.061711,877.164783,29461.320744,4.059732,-419174.204427,52935.804542


In [13]:
# double check for repeats!
np.sum(trainDF.iloc[:, [16,17,18]].duplicated()) # 0 means no repeats

0

In [14]:
print(trainDF.shape)
trainDF.head()

(10000000, 20)


,x_0,xd_0,y_0,yd_0,theta_0,thetad_0,phi_0,phid_0,x_f,xd_f,y_f,yd_f,theta_f,thetad_f,phi_f,phid_f,F,alpha,tau0,tau_w
0,0.0,1473.201644,0.0,1276.409045,3.623862,-11.851729,1.211000,7.662472,29.866409,564.269592,21.336507,178.728114,-10.007981,-1511.665595,-11.460196,-1484.165029,41181.992912,1.987846,-632162.376646,-59087.944289
1,0.0,292.053762,0.0,329.129735,0.473141,-13.332132,0.030206,-24.548492,7.249558,-18.500791,4.674845,811.994233,10.193245,869.755012,8.614343,801.903597,25150.218788,3.741918,929029.039471,30635.419374
2,0.0,122.864494,0.0,-77.860175,2.080511,-23.745955,2.269162,13.577434,0.795119,2307.996013,-0.387013,-1210.224936,13.660000,2437.064176,13.518852,2393.363053,33176.564043,4.106501,495429.618543,92261.347215
3,0.0,321.464802,0.0,-185.568646,5.970777,17.674272,0.253450,-23.262211,3.218772,742.740898,-2.445868,-238.234357,7.546394,436.597363,2.136279,479.836917,371.601599,0.668810,-402592.572461,31282.236616
4,0.0,-74.555675,0.0,1242.566503,1.493441,7.679440,1.916887,10.558238,-1.249155,1452.849870,18.110278,808.550119,13.904002,1641.289100,13.440743,1565.438974,35874.696078,5.480043,929295.194744,44737.069384


In [15]:
# rename columns to be consistent with other code
columnMapping = {"x0" : "x_0", "y0" : "y_0", "phi0" : "phi_0", "theta0" : "theta_0", 
                        "x_f" : "x_99", "y_f" : "y_99", "phi_f" : "phi_99", "theta_f" : "theta_99", 
                        "xd_0" : "x_dot_0", "yd_0" : "y_dot_0", "phid_0" : "phi_dot_0", "thetad_0": "theta_dot_0", 
                        "xd_f" : "x_dot_99", "yd_f": "y_dot_99", "phid_f": "phi_dot_99", "thetad_f": "theta_dot_99", 
                        "tau0" : "tau"}

# rename columns
trainDF.rename(columns=columnMapping, inplace=True)
testDF.rename(columns=columnMapping, inplace=True)


trainDF.head()
testDF.head()

,x_0,x_dot_0,y_0,y_dot_0,theta_0,theta_dot_0,phi_0,phi_dot_0,x_99,x_dot_99,y_99,y_dot_99,theta_99,theta_dot_99,phi_99,phi_dot_99,F,alpha,tau,tau_w
0,0.0,-479.406169,0.0,808.125168,1.074911,14.604692,1.610820,0.342567,-6.660783,606.471232,12.261791,621.515054,-8.236292,-1336.864852,-7.196589,-1285.116125,34939.199287,2.388669,-398848.635711,-52170.359464
1,0.0,730.738390,0.0,1220.929629,0.827197,-23.387488,4.311270,-0.881112,14.569797,-184.936022,24.523712,615.494066,17.904582,1735.428823,21.279341,1724.809150,3107.102381,1.483513,-25964.162554,91746.061771
2,0.0,-535.832562,0.0,11.137485,4.688616,23.204799,2.610169,4.804531,-10.027311,-496.451379,0.622225,34.844611,3.955242,-59.962069,2.982034,-5.562360,4516.937271,3.652675,-669707.091885,-1741.998064
3,0.0,175.278845,0.0,785.341568,5.317840,2.127405,2.094728,-20.938639,4.916500,-234.110480,13.478325,-17.188555,18.349930,1540.056108,15.515905,1555.480968,32299.290613,0.430122,-255553.079201,90618.166726
4,0.0,-817.253749,0.0,1121.004442,5.870598,3.003811,1.970784,5.114152,-16.960388,-378.720359,18.915203,935.576479,14.181347,839.473717,11.061711,877.164783,29461.320744,4.059732,-419174.204427,52935.804542


In [16]:
# convert to fx and fy
trainDF["Fx"] = trainDF.F * np.cos(trainDF.alpha)
trainDF["Fy"] = trainDF.F * np.sin(trainDF.alpha)

# convert to fx and fy
testDF["Fx"] = testDF.F * np.cos(testDF.alpha)
testDF["Fy"] = testDF.F * np.sin(testDF.alpha)

In [17]:
# make dataset
Xtrain = trainDF.loc[:, [ "phi_0", "theta_0", 
                   "x_dot_0", "y_dot_0", "phi_dot_0", "theta_dot_0",
                   "x_99", "y_99", "phi_99", "theta_99"]]

Ytrain = trainDF.loc[:, ["Fx", "Fy", "tau", "tau_w", "x_dot_99", "y_dot_99", "phi_dot_99", "theta_dot_99",] ]

# make dataset
Xtest = testDF.loc[:, Xtrain.columns]

Ytest = testDF.loc[:, Ytrain.columns ]


In [18]:
Xtrain.head()

,phi_0,theta_0,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,x_99,y_99,phi_99,theta_99
0,1.211000,3.623862,1473.201644,1276.409045,7.662472,-11.851729,29.866409,21.336507,-11.460196,-10.007981
1,0.030206,0.473141,292.053762,329.129735,-24.548492,-13.332132,7.249558,4.674845,8.614343,10.193245
2,2.269162,2.080511,122.864494,-77.860175,13.577434,-23.745955,0.795119,-0.387013,13.518852,13.660000
3,0.253450,5.970777,321.464802,-185.568646,-23.262211,17.674272,3.218772,-2.445868,2.136279,7.546394
4,1.916887,1.493441,-74.555675,1242.566503,10.558238,7.679440,-1.249155,18.110278,13.440743,13.904002


In [19]:
Ytrain.head()

,Fx,Fy,tau,tau_w,x_dot_99,y_dot_99,phi_dot_99,theta_dot_99
0,-16681.383440,37652.197635,-632162.376646,-59087.944289,564.269592,178.728114,-1484.165029,-1511.665595
1,-20752.753856,-14207.628673,929029.039471,30635.419374,-18.500791,811.994233,801.903597,869.755012
2,-18893.805673,-27271.019579,495429.618543,92261.347215,2307.996013,-1210.224936,2393.363053,2437.064176
3,291.543841,230.412537,-402592.572461,31282.236616,742.740898,-238.234357,479.836917,436.597363
4,24913.148195,-25813.346657,929295.194744,44737.069384,1452.849870,808.550119,1565.438974,1641.289100


In [20]:
# scale data 
scalerX = MinMaxScaler([-0.5, 0.5])  
scalerY = MinMaxScaler([-0.5, 0.5])  

# Don't cheat - fit only on training data
scalerX.fit(Xtrain)  
scalerY.fit(Ytrain) 

Xtrain_scaled = scalerX.transform(Xtrain)  
Ytrain_scaled = scalerY.transform(Ytrain)  

# apply same transformation to test data
Xtest_scaled = scalerX.transform(Xtest)
Ytest_scaled = scalerY.transform(Ytest)

In [21]:
pd.DataFrame(Xtrain_scaled, columns = Xtrain.columns).head()

,phi_0,theta_0,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,x_99,y_99,phi_99,theta_99
0,-0.307263,0.076756,0.491067,0.425470,0.153250,-0.237035,0.388537,0.282763,-0.296225,-0.270092
1,-0.495193,-0.424697,0.097351,0.109710,-0.490970,-0.266643,0.094132,0.065258,0.107981,0.142941
2,-0.138852,-0.168876,0.040955,-0.025953,0.271549,-0.474919,0.010114,-0.000821,0.206735,0.213822
3,-0.459662,0.450279,0.107155,-0.061856,-0.465244,0.353485,0.041663,-0.027698,-0.022456,0.088824
4,-0.194918,-0.262311,-0.024852,0.414189,0.211165,0.153589,-0.016496,0.240647,0.205162,0.218811


In [22]:
pd.DataFrame(Ytrain_scaled, columns = Ytrain.columns).head()

,Fx,Fy,tau,tau_w,x_dot_99,y_dot_99,phi_dot_99,theta_dot_99
0,-0.188277,0.424985,-0.316081,-0.295440,0.067071,0.019699,-0.256928,-0.259961
1,-0.234233,-0.160366,0.464515,0.153177,-0.002126,0.095052,0.140751,0.155220
2,-0.213250,-0.307815,0.247715,0.461307,0.274120,-0.145573,0.417597,0.428467
3,0.003310,0.002598,-0.201296,0.156411,0.088263,-0.029915,0.084725,0.079702
4,0.281233,-0.291362,0.464648,0.223685,0.172580,0.094642,0.273574,0.289730


# Keras models

In [23]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

In [24]:
K.clear_session()

In [25]:
# refref: start with small network and then build up
# refref: start with large network and prune

In [26]:
# create network
def create_network(optimizer = 'rmsprop', 
                    numUnits = [32, 32, 32, 32], 
                    weightRegularization = 0.0, 
                    dropout_rate=0.1):
    
    '''
    Create a feed forward network.  Assumes Xtrain & Ytrain have been created and scaled
    
    Params: 
    optimizer (str): choice of optimizer
    numUnits (list): number of units in each hidden
    weightRegularization (float): between 0 and 1
    dropout_rate (float): between 0 and 1
    
    '''
    K.clear_session()
    inputs = Input(shape=(Xtrain_scaled.shape[1],))    
    
    # add layers
    for ii in np.arange(0, len(numUnits)):
        if ii >= 1: 
            x = Dense(numUnits[ii], activation='tanh', 
                      kernel_regularizer=regularizers.l1(weightRegularization))(x)

        else: 
            x = Dense(numUnits[ii], activation='tanh')(inputs)


        # add dropout
        if dropout_rate > 0: 
            x = Dropout(dropout_rate)(x)
    
    predictions = Dense(Ytrain_scaled.shape[1], activation='linear')(x)

    # create model
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss="mean_squared_error", optimizer = optimizer, metrics = ['mse'])

    return(model)

In [35]:
modelParams = {"optimizer": "rmsprop", 
              "dropout_rate" : 0, 
               "numUnits": [400, 400, 400, 16],
               "weightRegularization": 0
              }


model = create_network(**modelParams)
modeltimestamp = datetime.now().strftime("%Y_%m_%d__%I_%M_%S")

modelName = ''.join('{}_{}__'.format(key[0:3].capitalize(), val) for  key, val in modelParams.items()).\
                            replace("[", "").replace("]", "").replace(", ", "_")[0:-2] + "_" + modeltimestamp
print(modelName)

model.summary()

Opt_rmsprop__Dro_0__Num_400_400_400_16__Wei_0_2019_11_14__12_32_34
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               4400      
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
dense_3 (Dense)              (None, 400)               160400    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                6416      
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
Total params: 331,752
Trainable params: 331,752
Non-trainable params: 0
___

In [36]:
# # save scalers, to be used on test set
scalerfileX = 'scalerX_fullact_' + modeltimestamp + '.pkl'
pickle.dump(scalerX, open(os.path.join(dataOutput, scalerfileX), 'wb'))

scalerfileY = 'scalerY_fullact_' + modeltimestamp + '.pkl'
pickle.dump(scalerY, open(os.path.join(dataOutput, scalerfileY), 'wb'))

In [72]:
#model.get_config()
earlystop = EarlyStopping(monitor='val_mean_squared_error', patience=150, 
                          verbose=1, mode='auto', min_delta = 0.000001)

In [38]:
# start training
historyDict = {"mean_squared_error": [], 
               "val_mean_squared_error": []}

In [ ]:
# # fit model without regularization
stt = time.time()
history = model.fit(Xtrain_scaled, Ytrain_scaled, epochs = 500, verbose = 2, 
                        batch_size=2**14, callbacks=[earlystop], validation_split = 0.3)
winsound.PlaySound("*", winsound.SND_ALIAS)
endd = time.time() - stt
print(endd)

Train on 7000000 samples, validate on 3000000 samples
Epoch 1/500
 - 9s - loss: 0.0065 - mean_squared_error: 0.0065 - val_loss: 0.0066 - val_mean_squared_error: 0.0066
Epoch 2/500
 - 9s - loss: 0.0064 - mean_squared_error: 0.0064 - val_loss: 0.0066 - val_mean_squared_error: 0.0066
Epoch 3/500
 - 9s - loss: 0.0064 - mean_squared_error: 0.0064 - val_loss: 0.0063 - val_mean_squared_error: 0.0063
Epoch 4/500
 - 9s - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0063 - val_mean_squared_error: 0.0063
Epoch 5/500
 - 9s - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0062 - val_mean_squared_error: 0.0062
Epoch 6/500


In [ ]:
historyDict["mean_squared_error"].extend(history.history["mean_squared_error"])
historyDict["val_mean_squared_error"].extend(history.history["val_mean_squared_error"])

In [ ]:
# save model and history
model.save(os.path.join(savedModels,  modelName + "_fullActuated" + '.h5'))
pickle.dump(historyDict, open(os.path.join(dataOutput, modelName + "history.pkl"), "wb"))

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(10,5))
axs.plot(historyDict["val_mean_squared_error"], label = "val", color = "C1")
axs.plot(historyDict["mean_squared_error"], label = "train", color = "C0")
axs.set_title('Model MSE = '+ str((historyDict['val_mean_squared_error'][-1]))[:8])
axs.set_ylabel('Mean squared error')
axs.set_xlabel('Epoch')
axs.legend( loc='best')
plt.yscale('log') #logarithmic scale for y axis
fig.savefig(os.path.join(figDir, modelName + '_training.png'))

___

## Evaluate accuracy on test data

___

In [ ]:
from keras.models import load_model

modelPath = os.path.join(savedModels,  modelName + "_fullActuated" + '.h5')

model = load_model(modelPath)

In [ ]:
# load scalers
scalerX = pickle.load(open(os.path.join(dataOutput, scalerfileX), "rb"))
scalerY = pickle.load(open(os.path.join(dataOutput, scalerfileY), "rb"))


In [ ]:

# predict
Ytest_pred = model.predict(Xtest_scaled)


# make data frames
XtestDF = pd.DataFrame(scalerX.inverse_transform(Xtest_scaled), columns = Xtrain.columns)
YtestDF = pd.DataFrame(scalerY.inverse_transform(Ytest_scaled), columns = Ytrain.columns)
YpredDF = pd.DataFrame(scalerY.inverse_transform(Ytest_pred), columns = Ytrain.columns+ "_pred")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df_c = pd.concat([YtestDF.reset_index(drop=True), YpredDF], axis=1)
df_c.head()



In [ ]:
# df_c = df_c.iloc[:1000, :]

In [ ]:
# make plots
from sklearn.metrics import r2_score
from matplotlib import ticker

M = 3
yticks = ticker.MaxNLocator(M)

plt.close("all")
fig, axs = plt.subplots(2,8, figsize=np.array([25, 9]), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.2, wspace=0.7)
#fig.suptitle('Predicted vs. acutal ', fontsize=14, fontweight='bold')

axs = axs.ravel("C")

ylabs = [r"g*cm/$s^2$", r"g*cm/$s^2$", r"g*$cm^2$/$s^2$", r"g*$cm^2$/$s^2$", "cm/s", "cm/s", "rad/s", "rad/s"]

# replace lightest colors with white
import matplotlib.colors

cmap = plt.cm.magma_r
cmaplist = np.array([cmap(i) for i in range(cmap.N)])
cmaplist[:,0:3] = np.divide(cmaplist[:, 0:3], 1.1)
cmaplist[0] = (1,1,1,0.5)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, cmap.N)

# Plot y = actual, x = predicted
for ii in np.arange(0, df_c.shape[1] //2):
    axs[ii].hexbin(y = df_c.loc[:,YtestDF.columns[ii]],
                   x = df_c.loc[:,YpredDF.columns[ii]], 
                   gridsize = 50, cmap = cmap)
    axs[ii].xaxis.set_major_locator(ticker.MaxNLocator(3))
    axs[ii].yaxis.set_major_locator(ticker.MaxNLocator(5))

    if(ii == 0):
        axs[ii].set_ylabel("Actual Value\n" + ylabs[ii])
    else:
        axs[ii].set_ylabel(ylabs[ii])

    axs[ii].set_title(YtestDF.columns[ii])
    axs[ii].plot(df_c.loc[:,YtestDF.columns[ii]], 
                 df_c.loc[:,YtestDF.columns[ii]], 
                 'grey', linewidth = 1, linestyle  = "--")

    # annotate with R^2
    axs[ii].text(np.max(df_c.loc[:,YtestDF.columns[ii]])*-0.2, 
                 np.min(df_c.loc[:,YtestDF.columns[ii]])*0.9, 
                 r'$r^2$ =' + 
                 str(np.round((r2_score(df_c.loc[:,YtestDF.columns[ii]],  
                                        df_c.loc[:,YpredDF.columns[ii]])), 3)))
    axs[ii].set_xlim([-np.max(np.abs(df_c.loc[:,YpredDF.columns[ii]])), 
                      np.max(np.abs(df_c.loc[:,YpredDF.columns[ii]]))])

# # residual plots x = predicted, y = actual - predicted
# jj is column in dataset, plotNum is plot position
for jj, plotNum in enumerate(np.arange(8, df_c.shape[1])):

    axs[plotNum].hexbin(y = df_c.loc[:,YtestDF.columns[jj]] - df_c.loc[:,YpredDF.columns[jj]],
                   x = df_c.loc[:,YpredDF.columns[jj]], 
                   gridsize = 50, cmap = cmap) 
    axs[plotNum].xaxis.set_major_locator(ticker.MaxNLocator(3))
    axs[plotNum].yaxis.set_major_locator(ticker.MaxNLocator(5))
    axs[plotNum].set_xlabel("Predicted Value\n" + ylabs[jj])

    if(jj == 0):
        axs[plotNum].set_ylabel("Actual - Predicted\n" + ylabs[jj])
    else:
        axs[plotNum].set_ylabel(ylabs[jj])

    axs[plotNum].hlines(y = 0, xmin = np.min( df_c.loc[:,YpredDF.columns[jj]]), 
                   xmax = np.max( df_c.loc[:,YpredDF.columns[jj]]), 
                   linestyle =  "--", linewidth = 1)
    axs[plotNum].set_ylim([-np.max(np.abs(df_c.loc[:,YtestDF.columns[jj]] - df_c.loc[:,YpredDF.columns[jj]])), 
                      np.max(np.abs(df_c.loc[:,YtestDF.columns[jj]]- df_c.loc[:,YpredDF.columns[jj]]))])


fig.savefig(os.path.join(figDir, "PredVActual_fullAct" + modelName + ".png"), dpi = 500, bbox_inches='tight')
plt.show()
# plt.close()